# Step-1: Required Modules 

In [ ]:
!python -m pip install pyyaml==5.1

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# Step 1: Install and import required libraries

import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

# Imports
import os
import cv2
import random
import torch
import numpy as np
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog



# Step 2: Register your dataset
![Alt text for the image](assets\image.png)

In [ ]:
# Step 2: Register your dataset

# Path to your main dataset directory
data_dir = "/content/drive/MyDrive/Sample Data"  # Change this to your dataset path


# Register the training dataset
register_coco_instances(
    "logo_train1",                         # Name for the dataset
    {},                                    # No additional metadata
    f"{data_dir}/Annotations/train.json",  # Path to annotation JSON
    f"{data_dir}/train/tshirt"             # Path to training images
)

# Register the validation dataset
register_coco_instances(
    "logo_val1",                           # Name for the validation dataset
    {},                                    # No additional metadata
    f"{data_dir}/Annotations/val.json",    # Path to annotation JSON
    f"{data_dir}/validation/val"           # Path to validation images
)


In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog
print("Train samples:", len(DatasetCatalog.get("logo_train1")))
print("Val samples:", len(DatasetCatalog.get("logo_val1")))


# Step 3: Configuration

In [ ]:
# Step 3: Configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("logo_train1",)
cfg.DATASETS.TEST = ("logo_val1",)
cfg.DATALOADER.NUM_WORKERS = 1

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 100  # increase this if needed
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # Set this correctly
cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


# Step 4: Start Training

In [ ]:
# Step 4: Start Training
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


# Testing 

In [ ]:
# Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

# Visualize
val_metadata = MetadataCatalog.get("logo_val1")
# image_path = f"{data_dir}/validation/val/tshirt_125.jpg"
# # replace with your image
image_path = f"/content/tshirt_032.jpg"
im = cv2.imread(image_path)
outputs = predictor(im)
print(outputs)

v = Visualizer(im[:, :, ::-1], metadata=val_metadata, scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.figure(figsize=(10,10))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis("off")
plt.show()
